In [49]:
#--utf-8---
import gzip
import shutil
import pandas as pd
import sys
import io
import json
import re
import time

import ast
import astunparse

In [10]:

















ile = gzip.open('/Users/yurunsong/Desktop/Data/rawCode/python-000000000000.csv.gz') 
pf = pd.read_csv(file, encoding="utf-8")

In [11]:
pf.head(10)

,repo_name,ref,path,copies,content
0,Axford/OpenSCADMachineDesignFramework,refs/heads/master,scripts/static.py,6,#!/usr/bin/env python\n\n# Perform static code...
1,ghandiosm/Test,refs/heads/master,addons/mail/models/mail_followers.py,26,# -*- coding: utf-8 -*-\n\nfrom openerp import...
2,pwarren/AGDeviceControl,refs/heads/master,agdevicecontrol/thirdparty/site-packages/linux...,17,# Copyright (c) 2001-2004 Twisted Matrix Labor...
3,idan/oauthlib,refs/heads/master,tests/oauth2/rfc6749/endpoints/test_metadata.py,1,# -*- coding: utf-8 -*-\nfrom oauthlib.oauth2 ...
4,melviso/beatle,refs/heads/master,beatle/plugin/tools/ast_explorer/res/_param.py,2,"# -*- coding: utf-8 -*-\n\n_param = [\n""16 16 ..."
5,lihf2015/python_exportemail,refs/heads/master,pytest/src/pytest/manage.py,1,#!/usr/bin/env python\nimport os\nimport sys\n...
6,kunallillaney/thunder,refs/heads/master,test/test_extraction_methods_block.py,8,import unittest\n\nfrom thunder import Thunder...
7,40223225/40223225-cdb-0622,refs/heads/master,static/Brython3.1.1-20150328-091302/Lib/unitte...,737,import unittest\nfrom unittest.mock import sen...
8,trangel/OPTpy,refs/heads/master,OPTpy/Abinit/utils.py,1,"from __future__ import print_function, divisio..."
9,Menooker/gem5_pcm,refs/heads/master,src/dev/x86/Cmos.py,69,# Copyright (c) 2008 The Regents of The Univer...


In [12]:
# Processing about attribute Copies
#
# sum(pf["copies"].tolist())/len(pf["copies"])
# [int(len(pf["copies"])/2)]
# print(pf["content"][pf["copies"]>5000])

In [13]:
# print(pf["content"][142799])

In [14]:
content = pf["content"][7]
print(content)

import unittest
from unittest.mock import sentinel, DEFAULT


class SentinelTest(unittest.TestCase):

    def testSentinels(self):
        self.assertEqual(sentinel.whatever, sentinel.whatever,
                         'sentinel not stored')
        self.assertNotEqual(sentinel.whatever, sentinel.whateverelse,
                            'sentinel should be unique')


    def testSentinelName(self):
        self.assertEqual(str(sentinel.whatever), 'sentinel.whatever',
                         'sentinel name incorrect')


    def testDEFAULT(self):
        self.assertTrue(DEFAULT is sentinel.DEFAULT)

    def testBases(self):
        # If this doesn't raise an AttributeError then help(mock) is broken
        self.assertRaises(AttributeError, lambda: sentinel.__bases__)


if __name__ == '__main__':
    unittest.main()



## Python structure

In [15]:
'''
Generate Abstract syntax Tree structure for Python code
'''

class pythonNode():
    def __init__(self, start, end, name, parent, indent):
        self.start =start
        self.end =end
        self.parent =parent
        self.indent =indent
        self.name = name
        self.child = []

In [16]:
startwith = ["from", "import", "class", "def", "for", "while", "if", "else", "elif"]

def restruct(content, start, end, parent, indent):
    subContent = content[start: end]
    pattern = "\n\s{" + str(indent) + "}[a-zA-Z@\'{3}_#0-9]"
    idx = [(m.start(0), m.end(0)) for m in re.finditer(pattern, subContent)]

    name = ""
    for ele in startwith:
        if subContent[2:].startswith(ele):
            name = ele

    if len(idx) == 1:
        return pythonNode(start, end, parent, name, indent)
    else:
        for i in range(len(idx)):
            if i == 0:
                tmp = pythonNode(parent.start, start+idx[0][0], name, parent, indent)  
                restruct(content, parent.start, start + idx[0][0], tmp, indent + 4)
            elif i == len(idx) - 1:
                tmp = pythonNode(start+idx[-1][0]+1, parent.end, name, parent, indent)
                restruct(content, start + idx[-1][0] + 1, parent.end, tmp, indent + 4)
            else:
                tmp = pythonNode(start+idx[i][0]+1,start+idx[i+1][0],name, parent, indent)
                restruct(content, start + idx[i][0] + 1, start + idx[i + 1][0], tmp, indent + 4)
            parent.child.append(tmp)
        return parent

In [17]:
root = pythonNode(0, len(content),"root", None, -1)

In [18]:
restruct(content,0, len(content), root, 0)

In [19]:
# print(len(root.child))
# print(content[root.child[-2].start: root.child[-2].end])

## Python Extract Methods

In [20]:
def search_methods(content):
    '''
        Search the Method structure and function code.
    '''
    idxs_method = [(m.start(0), m.end(0)) for m in re.finditer(r"(?!\n)\s*def\s.*\(.*\):",content)]
    methodRange = []
    for i in idxs_method:
        subContent = content[i[0]:i[1]]
        indent  = num_of_Indent(subContent)
        position = find_next_indent(content[i[1]:],indent)
        methodRange.append((i[0],i[1]+position))
    return methodRange

In [21]:
def num_of_Indent(line):
    """
        return the number of indent in the line.
    """
    indent = 0
    while line[indent] == " ":
        indent += 1
    return indent

In [22]:
def find_next_indent(content, indent):
    '''
        find next position where the indent is higher in the content.
    '''
    start = 0
    while start < len(content):
        if content[start] == "\n":
            nextIndent = start+1+indent
            if content[nextIndent: nextIndent+2] != "  ":
                return start+1
        start += 1
    return start  

In [23]:
def clean_up_methods(content):
    '''
        Clean up extracted methods
    '''
    indent = num_of_Indent(content)
    replacement = "\n"+(" "*indent)
    content = content.replace(replacement, "\n") 
    return content.strip().strip("\n")

In [24]:
def clean_up_comments(content):
    '''
    Collect Description and DocString
    '''
    content = re.sub(r"\s*#\s.*\n","\n", content)
    content = re.sub(r"[\"]{3}\n\s*.*\n\s*[\"]{3}","", content)
    content = re.sub(r"\n{2,}","\n",content)
    return content


def get_descrip_from_method(content):
    idx = content.find("\n")
    name = content[:idx]
    descriptions = re.findall(r"(#\s.*\n)+",content)
    docString = re.findall(r"[\"]{3}\n\s*.*\n\s*[\"]{3}",content)
    name = name.replace("def ","")
    name = re.sub("\(.*\):", "", name)
    descriptions = [i.strip("#").strip() for i in descriptions]
    docString = [i.strip("\"").strip("\n").strip() for i in docString]
    return name, descriptions, docString

In [25]:
out = search_methods(content)

In [26]:
methods = [content[i[0]:i[1]] for i in out]

In [27]:
for i in methods:
    print(clean_up_methods(i))
    print("=====================================================")

def testSentinels(self):
    self.assertEqual(sentinel.whatever, sentinel.whatever,
                     'sentinel not stored')
    self.assertNotEqual(sentinel.whatever, sentinel.whateverelse,
                        'sentinel should be unique')
def testSentinelName(self):
    self.assertEqual(str(sentinel.whatever), 'sentinel.whatever',
                     'sentinel name incorrect')
def testDEFAULT(self):
    self.assertTrue(DEFAULT is sentinel.DEFAULT)
def testBases(self):
    # If this doesn't raise an AttributeError then help(mock) is broken
    self.assertRaises(AttributeError, lambda: sentinel.__bases__)


In [28]:
out = get_descrip_from_method(clean_up_methods(methods[1]))
print(out)
print()
print("Descriptions")
for i in out[1]:
    print(i)
print()
for i in out[2]:
    print(i)
print()
print("======================================")
print(clean_up_comments(clean_up_methods(methods[1])))

('testSentinelName', [], [])

Descriptions


def testSentinelName(self):
    self.assertEqual(str(sentinel.whatever), 'sentinel.whatever',
                     'sentinel name incorrect')


In [29]:
def processing_methods(contents):
    '''
        Procesing the methods and format to dict before storing in json. 
    '''
    data = []
    for i, body in enumerate(contents):
        if str(body) == "nan":
            continue
        output = searchMethods(body)
        methods = [body[i[0]:i[1]] for i in output]
        for method in methods:
            docs = getDescription(cleanUpMethods(method))
            func = cleanUpComments(cleanUpMethods(method))
            if len(docs[2]) > 0:
                data.append({
                    "name": docs[0],
                    "descrip": " ".join(docs[1]),
                    "docString": " ".join(docs[2]),
                    "function": func,    
                })
    return data


In [30]:
def store_to_json(data):
    '''
        Store into json file.
    '''
    with open('data.json', 'w') as fw:
        json.dump(data,fw)

In [53]:
# start = time.time()
# data = processing_methods(pf["content"])
# print(time.time()-start)

In [54]:
# store(data)

In [35]:
funcs = pd.read_json(sys.path[0]+"/data.json", orient='records')

In [36]:
funcs.head()

,name,descrip,docString,function
0,elapsedFunc,,1!,def elapsedFunc(self):\n
1,test_nmf,"order is irrelevant, but one of these must be ...",(BlockMethod) nmf with defaults,def test_nmf(self):\n \n tsc = ThunderCo...
2,test_sima,NOTE: this test was brittle and failed non-det...,(BlockMethod) with SIMA strategy,def test_sima(self):\n \n import sima.se...
3,formatter,,register formatters,def formatter(func):\n \n FORMATTERS[get...
4,url,,convert FROM dict OR string TO URL PARAMETERS,def url(value):\n \n return value2url_pa...


In [44]:
print(f'function name: {funcs["name"][10]}')
print("--"*10)
print(f'DocString: {funcs["docString"][10]}')
print("--"*10)
print(f'Function Body:\n{funcs["function"][10]}')

function name: utf82unicode
--------------------
DocString: WITH EXPLANATION FOR FAILURE
--------------------
Function Body:
def utf82unicode(value):
    
    try:
        return value.decode("utf8")
    except Exception as e:
        if not _Log:
            _late_import()


## Natural Languages

In [45]:
def getNL(content):
    '''
        Get the Docstrings, Descriptions, class and function name, 
        which are used as pretraining Natural Language input.
    '''
    
    output = []
    output_idx = []
    
    idx_class = [(m.start(0), m.end(0)) for m in re.finditer(r"(?!\n)\s*class\s.*\(.*\):",content)]
    idx_name = [(m.start(0), m.end(0)) for m in re.finditer(r"(?!\n)\s*def\s.*\(.*\):",content)]
    idx_descrip = [(m.start(0), m.end(0)) for m in re.finditer(r"(#\s.*\n)+",content)]    
    idx_docString = [(m.start(0), m.end(0)) for m in re.finditer(r"[\"]{3}\n?\s*.*\n?\s*[\"]{3}",content)]

    
    output_idx.extend(idx_class)
    output_idx.extend(idx_name)
    output_idx.extend(idx_descrip)
    output_idx.extend(idx_docString)
    
    for i in sorted(output_idx):
        code_str = content[i[0]:i[1]]
        code_str = code_str.strip("#").strip('"\"').strip("\n")
        code_str = code_str.strip()
        output.append(code_str)
    
    return output

In [46]:
getNL(content)

['class SentinelTest(unittest.TestCase):',
 'def testSentinels(self):',
 'def testSentinelName(self):',
 'def testDEFAULT(self):',
 'def testBases(self):',
 "If this doesn't raise an AttributeError then help(mock) is broken"]

## ControlFlow

In [47]:
code = funcs["function"][21]

In [50]:
node = ast.parse(code)
print(ast.dump(node))
print(astunparse.unparse(node))

Module(body=[FunctionDef(name='remove_profile_images', args=arguments(args=[arg(arg='profile_image_names', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='storage', ctx=Store())], value=Call(func=Name(id='get_profile_image_storage', ctx=Load()), args=[], keywords=[])), For(target=Name(id='name', ctx=Store()), iter=Call(func=Attribute(value=Name(id='profile_image_names', ctx=Load()), attr='values', ctx=Load()), args=[], keywords=[]), body=[Expr(value=Call(func=Attribute(value=Name(id='storage', ctx=Load()), attr='delete', ctx=Load()), args=[Name(id='name', ctx=Load())], keywords=[]))], orelse=[])], decorator_list=[], returns=None)])


def remove_profile_images(profile_image_names):
    storage = get_profile_image_storage()
    for name in profile_image_names.values():
        storage.delete(name)



In [51]:
def CodeVisitor(node, indent):
    '''
        Recursively visit AST node with indent for a method.
        Only stmt, excepthandler are in the consideration.
        Other types like expr, boolop, operator, unaryop, arguments are not.
        
    '''
    
    if sum(1 for x in ast.iter_child_nodes(node)) == 0:
        return
    
    else:
        if isinstance(node, ast.stmt):
            print(f"{indent} {type(node).__name__}")


        if isinstance(node, ast.excepthandler):
            indent -= 1
            print(f"{indent} {type(node).__name__}")
            
            
        if "orelse" in node._fields and isinstance(node, ast.If) and len(node.orelse) > 0 :
            
            CodeVisitor(node.test, indent)
            indent += 1
            for j in node.body:
                CodeVisitor(j, indent)

            indent -= 1 
            print(f"{indent} else")
            
            indent += 1
            for k in node.orelse:
                CodeVisitor(k, indent)
                    
        else:
            indent += 1
            for i in ast.iter_child_nodes(node):
                CodeVisitor(i, indent)


In [52]:
print(code)
print("=============================")
node = ast.parse(code)
CodeVisitor(node, -1)

def remove_profile_images(profile_image_names):
    
    storage = get_profile_image_storage()
    for name in profile_image_names.values():
        storage.delete(name)
0 FunctionDef
1 Assign
1 For
2 Expr
